In [ ]:
!pip install -U tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_csv("sample_data/spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [ ]:
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Message'],df['spam'], stratify=df['spam'])

In [ ]:
X_train.head(4)

1239    Dear relieved of westonzoyland, all going to p...
97      i see. When we finish we have loads of loans t...
3861               Yep. I do like the pink furniture tho.
813     Congratulations ur awarded either £500 of CD g...
Name: Message, dtype: object

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351724, -0.51327276, -0.88845724, ..., -0.74748844,
        -0.7531474 ,  0.91964495],
       [-0.87208337, -0.5054394 , -0.94446665, ..., -0.85847497,
        -0.71745324,  0.8808295 ]], dtype=float32)>

In [ ]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [ ]:
cosine_similarity([e[0]],[e[3]])

array([[0.84703803]], dtype=float32)

In [ ]:
cosine_similarity([e[3]],[e[4]])

array([[0.9872035]], dtype=float32)

In [ ]:
with tf.device('/device:GPU:0'):
  # Bert layers
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessed_text = bert_preprocess(text_input)
  outputs = bert_encoder(preprocessed_text)

  # Neural network layers
  l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
  l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

  # Use inputs and outputs to construct a final model
  model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
with tf.device('/device:GPU:0'):
  model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
len(X_train)

4179

In [ ]:
with tf.device('/device:GPU:0'):
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
  model.fit(X_train, y_train, epochs=5)

Epoch 1/5
131/131 [==============================] - 51s 318ms/step - loss: 0.3527 - accuracy: 0.8624
Epoch 2/5
131/131 [==============================] - 42s 323ms/step - loss: 0.2500 - accuracy: 0.8885
Epoch 3/5
131/131 [==============================] - 42s 318ms/step - loss: 0.2066 - accuracy: 0.9208
Epoch 4/5
131/131 [==============================] - 42s 319ms/step - loss: 0.1826 - accuracy: 0.9294
Epoch 5/5
131/131 [==============================] - 42s 321ms/step - loss: 0.1648 - accuracy: 0.9450


In [ ]:
with tf.device('/device:GPU:0'):
  model.evaluate(X_test, y_test)

44/44 [==============================] - 15s 312ms/step - loss: 0.1452 - accuracy: 0.9397


In [ ]:
reviews = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)

1/1 [==============================] - 1s 735ms/step


array([[0.45785826],
       [0.55507994],
       [0.37880445],
       [0.03758238],
       [0.01758032]], dtype=float32)

In [ ]:
model.save("saved_models/1/")

In [ ]:
!zip -r /content/file.zip /content/saved_models/

  adding: content/saved_models/ (stored 0%)
  adding: content/saved_models/1/ (stored 0%)
  adding: content/saved_models/1/keras_metadata.pb (deflated 84%)
  adding: content/saved_models/1/saved_model.pb (deflated 92%)
  adding: content/saved_models/1/variables/ (stored 0%)
  adding: content/saved_models/1/variables/variables.index (deflated 79%)
  adding: content/saved_models/1/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/saved_models/1/assets/ (stored 0%)
  adding: content/saved_models/1/assets/vocab.txt (deflated 53%)


In [ ]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>